In [ ]:
%load_ext autoreload

In [ ]:
%autoreload

import h2o
import create_data as create
import pandas as pd
import create_delivery_file as delivery
import save_model as save

from h2o.automl import H2OAutoML

In [ ]:
data = create.CreateTrainingData()
train = data.create_training_data_with_mean_features()
test = create.create_test_data()
target = create.create_target_data()

In [ ]:
h2o.init()

In [ ]:
# Delete all rows with nans in pv_measurement column in train
# train = train[train['pv_measurement'].isna() == False]

X_train = h2o.H2OFrame(train)
y = 'pv_measurement'

X = X_train.columns
X.remove(y)

In [ ]:
autoML = H2OAutoML(max_models=30, seed=4, nfolds=5)
autoML.train(x=X, y=y, training_frame=X_train)

In [ ]:
best_model = autoML.leader
save.save_model(best_model, 'h2o_mean')

In [ ]:
# Remove features that are not in the training set
# test.fillna(0, inplace=True)
test_pd = test[X]
test_h2o = h2o.H2OFrame(test_pd)

pred = autoML.predict(test_h2o)

In [ ]:
file = delivery.CreateDelivery(pred, 'mean_features_in_training_no_feature_engineering')
file.create_delivery_file()